# Loading 2D Annotations

In [ ]:
from src.loading import Coco
rootdir = '/path/to/scout/'
coco = Coco(rootdir, sequence =1)
# use `coco_dict = coco.load()` to gain the full coco annotations
img = coco.annotate_frame(0, 'cam_0')
coco.show_frame(img)


In [ ]:
# Annotations loaded only for single frame and camera
from src.loading import Individual
rootdir = '.'
indiv = Individual(rootdir, sequence =1)
img = indiv.annotate_frame(81, 'cam_2')
indiv.show_frame(img)

In [ ]:
from src.loading import MOT
rootdir = '.'
mot = MOT(rootdir, sequence =1)
img = mot.annotate_frame(0, 'cam_0')
mot.show_frame(img)

In [ ]:

from src.loading import AnnotationDataset, collate_fn, Individual
from torch.utils.data import DataLoader

loader = Individual(root_dir=".", sequence=1)
frame_ids = list(range(100))           # or your custom list
cam_ids =  ["cam_0", "cam_2"]# or any available cams

dataset = AnnotationDataset(loader, frame_ids, cam_ids)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn, num_workers=4)

In [ ]:
from src.visualisation import draw_annotation
import matplotlib.pyplot as plt
import cv2
import numpy as np

batch = next(iter(dataloader))
image = (batch['images'][0].numpy().transpose(1, 2, 0) * 255.0).astype(np.uint8)
cam_id = batch['cam_ids'][0]
frame_id = batch['frame_ids'][0]
annotations = batch['bboxes'][0]
track_ids = batch['track_ids'][0]
timestamp = batch['timestamps'][0]

for i, bbox in enumerate(annotations):
    image = draw_annotation(image, bbox, track_ids[i])

# Show
plt.figure(figsize=(8, 6))
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title(f"Cam: {cam_id}, Frame: {frame_id}, Timestamp: {timestamp}s")
plt.axis('off')
plt.show()

# 3D Mesh Loading

In [ ]:
import trimesh
from pathlib import Path
rootdir = "."
mesh_paths = Path(rootdir) / 'dataset' / 'mesh'
high_resolution_mesh = trimesh.load(mesh_paths / 'high_resolution_mesh.ply')
low_resolution_mesh = trimesh.load(mesh_paths / 'low_resolution_mesh.ply')

# Visualising Predictions

downloaded:
    - dataset
        - mesh
        - images
        



In [ ]:
from src.visualisation import draw_3d
from src.geometry import bbox_to_mesh
from src.loading import load_calibration, Individual
import numpy as np

rootdir = '.'
cam_name = 'cam_0'
calibration = load_calibration('.', cam_name, sequence = 1)
indiv = Individual(rootdir, sequence =1)


frame_id = 100

annotation = indiv.retrieve(frame_id, cam_name)[0]
projected_3d = bbox_to_mesh(annotation['bbox'], calibration, high_resolution_mesh)
world_coordinate = np.array([annotation['world']['Xw'], annotation['world']['Yw'], annotation['world']['Zw']])
draw_3d(np.stack([world_coordinate, projected_3d[0]]), low_resolution_mesh)